# Import modules

In [1]:
!pwd
import pandas as pd
import json
import ipycytoscape


blast_data_con = pd.read_csv('files/Blaste40_out', sep='\t', header=None)

headerList = ['source','target','evalue']

blast_data_con.to_csv('files/Blaste40_out_2', header=headerList, index=False, sep='\t')

blast_data_con = pd.read_csv('files/Blaste40_out_2', sep='\t')

/Users/novakw/Box/_novakw/Big Data/my-first-binder/current working


In [2]:
#blast_data_con
import numpy as np

df = blast_data_con

#remove duplicates
m=pd.DataFrame(np.sort(df[['source','target']])).duplicated()
df = df[~m]
#removes self-reference
df = df[df.source != df.target]
edges = df
#edges
#edges.to_csv('files/edges_40', header=headerList, index=False, sep='\t')

lets convert this to a list of nodes and a list of connections

In [3]:
#nodes = df.drop_duplicates('source')
uniq_list = np.unique(df[['source', 'target']].values)
#print(uniq_list)
nodes = pd.DataFrame(uniq_list, columns = ['id'])
#nodes = nodes.drop(columns=['target', 'evalue'])
#nodes.columns=['id']

nodes['label'] = nodes['id']
nodes['background-color']='cyan'

#nodes.loc['3HRT_SCAR']
nodes['width']='24'
nodes['height']='24'
nodes['text-valign']='center'
nodes['text-halign']='center'

nodes['count'] = '1'

col_one_list = nodes['id'].tolist()

for item in col_one_list: 
    #print(item)
    size = len(edges[edges['source']==item]) + len(edges[edges['target']==item])
    nodes.loc[nodes['id'] == item, 'count']=str(size)
    size = size*10
    nodes.loc[nodes['id'] == item, 'width']=str(size)
    nodes.loc[nodes['id'] == item, 'height']=str(size)

records = edges.to_records(index=False)
result = list(records)
#print(result)
    
#edge_tar_list = edges['target'].tolist()

#edge_tar_list = edges.iterrows().tolist()

for item in result:
    #print(item)
    if item[1] == '1U8R_IDER':
        nodes.loc[nodes['id'] == item[0],'background-color']  = 'red'
    if item[1] == '5CVI_SLOR':
        nodes.loc[nodes['id'] == item[0],'background-color']  = 'blue'
    if item[1] == '3HRT_SCAR':
        nodes.loc[nodes['id'] == item[0],'background-color']  = 'green'
    if item[1] == '1C0W_DTXR':
        nodes.loc[nodes['id'] == item[0],'background-color']  = 'orange'
    if item[1] == '3R60_MNTR':
        nodes.loc[nodes['id'] == item[0],'background-color']  = 'magenta'
    if item[1] == '6O5C_MTSR':
        nodes.loc[nodes['id'] == item[0],'background-color']  = 'yellow'

nodes.loc[nodes['id'] == '1U8R_IDER','background-color']  = 'red'
nodes.loc[nodes['id'] == '1C0W_DTXR','background-color']  = 'orange'
nodes.loc[nodes['id'] == '6O5C_MTSR','background-color']  = 'yellow'
nodes.loc[nodes['id'] == '3HRT_SCAR','background-color']  = 'green'
nodes.loc[nodes['id'] == '5CVI_SLOR','background-color']  = 'blue'
nodes.loc[nodes['id'] == '3R60_MNTR','background-color']  = 'magenta'
    
nodes
#nodes.to_csv('files/nodes_40', index=True, sep='\t')

,id,label,background-color,width,height,text-valign,text-halign,count
0,A0A0E3NJG9,A0A0E3NJG9,cyan,20,20,center,center,2
1,A0A0F4KXT9,A0A0F4KXT9,cyan,80,80,center,center,8
2,A0A0F7PCE6,A0A0F7PCE6,cyan,30,30,center,center,3
3,A0A0F9JAM9,A0A0F9JAM9,cyan,10,10,center,center,1
4,A0A0F9UKL5,A0A0F9UKL5,cyan,30,30,center,center,3
...,...,...,...,...,...,...,...,...
255,R9SKE6,R9SKE6,cyan,10,10,center,center,1
256,S0EWQ7,S0EWQ7,cyan,100,100,center,center,10
257,T0I0U1,T0I0U1,cyan,30,30,center,center,3
258,U2USR3,U2USR3,cyan,40,40,center,center,4


In [4]:
def transform_into_ipycytoscape(nodes_df,edges_df):
    
    nodes_dict = nodes_df.to_dict('records')
    edges_dict = edges_df.to_dict('records')
    """   
    for thing in edges_dict:
        if thing['target'] == "1U8R_IDER":
            print(thing["source"])
    """
    # building nodes

    data_keys = ['id']  #this is a list of keys in stations (nodes)
    position_keys = ['position_x','position_y']
    rest_keys = ['score','idInt','name','score','group','removed','selected','selectable','locked','grabbed'
                 'grabbable']
    
    nodes_graph_list=[] #an empty list for making the json-like? file
    for node in nodes_dict: #iterating over each node
        dict_node = {}
        data_sub_dict = {'data':{el:node[el] for el in data_keys}}
        rest_sub_dict = {el:node[el] for el in node.keys() if el in rest_keys}
        posi_sub_dict = {}
        if 'position_x' in node.keys() and 'position_y' in node.keys():
            #print(node.keys())
            posi_sub_dict = {'position':{el:node[el] for el in node.keys() if el in position_keys}}
        
        dict_node = {**data_sub_dict,**rest_sub_dict,**posi_sub_dict}
        nodes_graph_list.append(dict_node)
    #print(nodes_graph_list) #NOTE this works to here!!!
    
    # building edges
    
    data_keys  = ['source','target','evalue'] #this is a list of keys in rails (edges)
    data_keys2 = ['label','classes'] #these are from the stations I think
    rest_keys  = ['score','weight','group','networkId','networkGroupId','intn','rIntnId','group','removed','selected','selectable','locked','grabbed','grabbable','classes']
    position_keys = ['position_x','position_y']
    
    edges_graph_list = []
    for edge in edges_dict:
        dict_edge = {}
        data_sub_dict = {el:edge[el] for el in data_keys}
        data_sub_dict2 = {el:edge[el] for el in edge.keys() if el in data_keys2}
        rest_sub_dict = {el:edge[el] for el in edge.keys() if el in rest_keys}
        
        dict_edge = {'data':{**data_sub_dict,**data_sub_dict2},**rest_sub_dict}
        edges_graph_list.append(dict_edge)
    
    #print(edges_graph_list)
    
    total_graph_dict = {'nodes': nodes_graph_list, 'edges':edges_graph_list}
    
    #print(total_graph_dict)
    
    # building the style
    all_node_style = ['background-color','background-opacity',
                     'font-family','font-size','label','width',
                     'shape','height','width','text-valign','text-halign']
    all_edge_style = ['background-color','background-opacity',
                     'font-family','font-size','label','width','line-color', 
                     ]
    
    total_style_dict = {}
    style_elements=[]
    for node in nodes_dict:
        node_dict = {'selector': f'node[id = \"{node["id"]}\"]'}
        style_dict ={"style": { el:node[el] for el in node.keys() if el in all_node_style}}
        node_dict.update(style_dict)
        style_elements.append(node_dict)
    
    for edge in edges_dict:
        edge_dict = {'selector': f'edge[id = \"{edge["source"]}\"]'}
        style_dict ={"style": { el:edge[el] for el in edge.keys() if el in all_edge_style}}
        edge_dict.update(style_dict)
        style_elements.append(edge_dict)
    
    # the graph
    data_graph = json.dumps(total_graph_dict)
    json_to_python = json.loads(data_graph)
    result_cyto = ipycytoscape.CytoscapeWidget()
    result_cyto.graph.add_graph_from_json(json_to_python)    
    result_cyto.set_style(style_elements)
    result_cyto.set_layout(name='grid')
    
    return result_cyto

G=transform_into_ipycytoscape(nodes,edges)
display(G)

{'nodes': [{'data': {'id': 'A0A0E3NJG9'}}, {'data': {'id': 'A0A0F4KXT9'}}, {'data': {'id': 'A0A0F7PCE6'}}, {'data': {'id': 'A0A0F9JAM9'}}, {'data': {'id': 'A0A0F9UKL5'}}, {'data': {'id': 'A0A0K2SKA5'}}, {'data': {'id': 'A0A0M9ECK3'}}, {'data': {'id': 'A0A0P6XKG5'}}, {'data': {'id': 'A0A0R1L5Q2'}}, {'data': {'id': 'A0A0R1M5P2'}}, {'data': {'id': 'A0A0R1V196'}}, {'data': {'id': 'A0A0R2FXZ6'}}, {'data': {'id': 'A0A0R2L1B1'}}, {'data': {'id': 'A0A0S8JFU7'}}, {'data': {'id': 'A0A0X8H0G0'}}, {'data': {'id': 'A0A0X8V1I5'}}, {'data': {'id': 'A0A101FFS4'}}, {'data': {'id': 'A0A117MHQ4'}}, {'data': {'id': 'A0A133VKD0'}}, {'data': {'id': 'A0A134CAN4'}}, {'data': {'id': 'A0A135VJJ3'}}, {'data': {'id': 'A0A136MYT5'}}, {'data': {'id': 'A0A136P3P6'}}, {'data': {'id': 'A0A150IWB6'}}, {'data': {'id': 'A0A178M510'}}, {'data': {'id': 'A0A1A0XES7'}}, {'data': {'id': 'A0A1C2B4G0'}}, {'data': {'id': 'A0A1C7E726'}}, {'data': {'id': 'A0A1E7HMH0'}}, {'data': {'id': 'A0A1E8GH58'}}, {'data': {'id': 'A0A1F2VCW0'}

CytoscapeWidget(cytoscape_layout={'name': 'grid'}, cytoscape_style=[{'selector': 'node[id = "A0A0E3NJG9"]', 's…